## Exercício: Modelo de Linguagem com auto-atenção

Este exercício é similar ao da aula passada, mas iremos agora treinar uma rede neural *com auto-atenção* para prever a próxima palavra de um texto, data as palavras anteriores como entrada.

Na camada de auto-atenção, deve-se implementar (vide slide 34):
- Embeddings de posição
- Projeções lineares (WQ, WK, WV, WO)
- Camada de feed forward (2-layer MLP)

Instrucões:
- É necessário fazer duas implementações da camada de auto-atenção: uma usando laços (ineficiente, mas fácil de entender) e outra matricial (eficiente mas difícil de entender). Usar slide 36 como referência.

- Fazer um assert para garantir que o resultado das duas implementações é exatamente igual.

- No treinamento, usar apenas a implementação matricial.

## Imports

In [1]:
import os
import sys
import random
import torch.nn as nn
import torch.nn.functional as F
import time
import math
from sklearn.model_selection import train_test_split
from torch import nn

## Variáveis Globais e Inicialização

In [2]:
# Global variables

# Vocabulary
vocab_size = 5000
context_size = 5
pattern = r'\w+|[,;.:!?\']'

# Training
batch_size = 128
epochs = 10
lr = 0.1

# Model
embedding_dim = 256
hidden_dim = 128

In [3]:
# Colab environment
IN_COLAB = 'google.colab' in sys.modules

if (IN_COLAB):
    %pip install colorama

    # Google Drive
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)

    project_folder="/content/drive/MyDrive/Classes/IA024/Aula_2_3"
    os.chdir(project_folder)
    !ls -la

## Faz download e carrega o dataset

In [4]:
# Check if download is necessary
if not os.path.exists("67724.txt.utf-8"):
    print("Downloading Gutenberg texts")

    !wget https://www.gutenberg.org/ebooks/67724.txt.utf-8
    !wget https://www.gutenberg.org/ebooks/67725.txt.utf-8

In [5]:
text = open("67724.txt.utf-8","r").read()
text += open("67725.txt.utf-8","r").read()

paragraphs = text.split("\n\n")

len(paragraphs)

4969

In [6]:
# Checking the text
print(paragraphs[0])

The Project Gutenberg eBook of O Guarany: romance brazileiro, Vol. 1 (of 2)
    
This ebook is for the use of anyone anywhere in the United States and
most other parts of the world at no cost and with almost no restrictions
whatsoever. You may copy it, give it away or re-use it under the terms
of the Project Gutenberg License included with this ebook or online
at www.gutenberg.org. If you are not located in the United States,
you will have to check the laws of the country where you are located
before using this eBook.


In [7]:
cleaned_paragraphs = [paragraph.replace("\n", " ") for paragraph in paragraphs if paragraph.strip()]

# Print 5 random paragraphs
num_paragraphs = len(cleaned_paragraphs)
for i in range(0,5):
    idx = random.randrange(num_paragraphs)
    print(f"{cleaned_paragraphs[idx]}\n")

print("Number of paragraphs: " + str(num_paragraphs))

len(cleaned_paragraphs)

--Precisas de Pery? disse elle.

--Mas, senhora, tu não vês que tuas mãos forão feitas para as flores e não para os espinhos; teus pés para brincar e não para andar; teu corpo para a sombra e não para o sol e a chuva?

--A vossa falta é daquellas que não se perdoão, disse D. Antonio; mas estamos nesse momento extremo em que Deus manda esquecer todas as offensas. Levantai-vos e preparemo-nos todos para morrer como christãos.

--Bem sabes que o aborreço!...

 XI

Number of paragraphs: 4892


4892

## Análise do dataset

In [8]:
# Conta as palavras no dataset
from collections import Counter
import re

def count_words(texts):
    word_counts = Counter()
    for text in texts:
        word_counts.update(re.findall(pattern, text.lower()))
    return word_counts

word_counts = count_words(cleaned_paragraphs)

len(word_counts)

12610

## Criando um vocabulário

In [9]:
most_frequent_words = [word for word, count in word_counts.most_common(vocab_size)]
vocab = {word: i for i, word in enumerate(most_frequent_words, 1)}

In [10]:
print(f"Most Frequent Words: {most_frequent_words[:10]}")
print(f"Vocabulary Size: {len(vocab)}")

Most Frequent Words: ['.', ',', 'a', 'que', 'o', 'de', 'e', 'se', ';', 'um']
Vocabulary Size: 5000


In [11]:
def encode_sentence(sentence, vocab):
    return [vocab.get(word, 0) for word in re.findall(pattern, sentence.lower())]

print(cleaned_paragraphs[20])
print(encode_sentence(cleaned_paragraphs[20], vocab))

 Publicando este livro em 1857, se disse ser aquella primeira edição uma prova typographica, que algum dia talvez o autor se dispuzesse a rever.
[0, 146, 4383, 23, 0, 2, 8, 50, 117, 276, 266, 2669, 13, 1071, 0, 2, 4, 193, 137, 287, 5, 2264, 8, 0, 3, 2672, 1]


## Classe do dataset

In [12]:
# Dataset class
import torch
from torch.utils.data import Dataset, DataLoader

class CustomDataset(Dataset):
  def __init__(self, paragraphs, vocab, context):
    self.paragraphs = paragraphs
    self.vocab = vocab
    self.context = context
    self.tokens, self.targets = self.setup()

  def __len__(self):
    return len(self.tokens)

  def __getitem__(self, idx):
    return torch.tensor(self.tokens[idx]), torch.tensor(self.targets[idx])
  
  def setup(self):
    tokens = []
    targets = []
    for paragraph in self.paragraphs:
      encoded = encode_sentence(paragraph, self.vocab)
      
      # If paragraph is smaller than the context, skip it.
      if len(encoded) < self.context + 1:
          continue

      for i in range(len(encoded) - self.context):
        tks = encoded[i:i+self.context]
        tgt = encoded[i+self.context]
        # Only add if there are no unknown tokens in both context and target.
        bad_token = 0
        if not (bad_token in tks or tgt == bad_token):
          tokens.append(tks)
          targets.append(tgt)
    return tokens, targets


In [13]:
# Train/Validation split
train_data, val_data = train_test_split(cleaned_paragraphs, test_size=0.2, random_state=18)

train_dataset = CustomDataset(train_data, vocab, context_size)
val_dataset = CustomDataset(val_data, vocab, context_size)

# Counting all Samples
print(f"Training samples: {len(train_data)}")
print(f"Validation samples: {len(val_data)}")
print()
print(f"Training dataset samples: {len(train_dataset)}")
print(f"Validation dataset samples: {len(val_dataset)}")

Training samples: 3913
Validation samples: 979

Training dataset samples: 59646
Validation dataset samples: 16215


In [14]:
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)

sample = next(iter(train_loader))
print(sample)

[tensor([[   4,   29, 1082,  229,    6],
        [   2,    7,   12,  314,   46],
        [   7,   93, 1606,    6, 3792],
        [   6,  854,    7,  556,  603],
        [   2,    7, 1471,    3, 2136],
        [  13, 3091,    9,  525,    8],
        [4703,   51,  473,    2,    7],
        [  14,  426,    2,    7,   77],
        [   2, 4616,    5,  749,    6],
        [ 868,  169,  736,   31,  421],
        [3997,  638, 1008, 4813,  537],
        [  22,  915,  277,    2,   50],
        [   2, 1769,   11,  337,    6],
        [ 384,  220,    5,   19, 2690],
        [  42, 3132,    2, 2254,    3],
        [ 923,   29,  193, 2549, 3574],
        [   4, 1346, 2921,    7, 1192],
        [ 156,    2,  167,  160,    9],
        [1915,    4, 2438,   38,   40],
        [3806,    2,   57,    2,    4],
        [ 631, 4184,    2,   50,   10],
        [  15, 1412,    6,   17,  696],
        [   1,    1,    1,    1,    1],
        [   1,    1,    1,    1,    1],
        [   9,  649,   15, 4261,  212],

## Model

#### Implementação das camadas de self-attention (Loop e Matricial)

In [15]:
# Positional Embedding - as described in "Attention is All You Need"
class PositionalEncoding(nn.Module):
    def __init__(self, max_sequence, embedding_dim):
        super().__init__()
        self.positional_encoding = torch.zeros(max_sequence, embedding_dim)
        
        # List of positions from 0..max_sequence in vector form
        position_list = torch.arange(0, max_sequence, dtype=torch.float)
        position_list = position_list.unsqueeze(1)

        div_term = torch.exp(torch.arange(0, embedding_dim, 2).float() * (-math.log(10000.0) / embedding_dim))

        self.positional_encoding[:, 0::2] = torch.sin(position_list * div_term)
        self.positional_encoding[:, 1::2] = torch.cos(position_list * div_term)
        self.positional_encoding = self.positional_encoding.unsqueeze(0)
    
    def forward(self, x):
        batch_size, seq_length, embedding_dim = x.size()
        positional_encoding = self.positional_encoding[:, :seq_length, :]
        positional_encoding = positional_encoding.to(x.device)
        # Position encoding is added to the input embeddings.
        return x + positional_encoding
    

In [16]:
# Adaptado da implementação do Ramon, muito elegante e traduziu bem o slide em aula.

class SelfAttention_Loop(nn.Module):
  def __init__(self, WQ, WK, WV, WO):
    super(SelfAttention_Loop, self).__init__()
    self.WQ = WQ
    self.WK = WK
    self.WV = WV
    self.WO = WO

  def forward(self, seq):
    E = []  
    for Xq in seq:
        q = self.WQ(Xq)
        scores = []
        for Xk in seq:
            k = self.WK(Xk)
            score = torch.dot(q, k.transpose(-1,0))
            scores.append(score)

        scores_tensor = torch.tensor(scores)  
        probs         = scores_tensor.softmax(dim=-1)

        e = 0
        for xv, p in zip(seq, probs):
            v = self.WV(xv)
            e += v * p

        e = self.WO(e)
        E.append(e)

    return torch.stack(E) 

In [28]:
# Matrix Implementation
class SelfAttention_Matrix(nn.Module):
  def __init__(self, embedding_dim):
    super().__init__()

    self.WQ = nn.Linear(embedding_dim, embedding_dim)
    self.WK = nn.Linear(embedding_dim, embedding_dim)
    self.WV = nn.Linear(embedding_dim, embedding_dim)
    self.WO = nn.Linear(embedding_dim, vocab_size)

  def forward(self, inputs):
    # Linear projections
    Q = self.WQ(inputs)
    K = self.WK(inputs)
    V = self.WV(inputs)

    scores = torch.matmul(Q, K.transpose(-2, -1))
    probs = F.softmax(scores, dim=-1)
    new_embedding = torch.matmul(probs, V)
    # Projection in WO
    new_embedding = self.WO(new_embedding)
    return new_embedding

# For testing purposes only
class SelfAttention_Matrix_tst(nn.Module):
  def __init__(self, WQ, WK, WV, WO):
    super(SelfAttention_Matrix_tst, self).__init__()
    self.WQ = WQ
    self.WK = WK
    self.WV = WV
    self.WO = WO

  def forward(self, inputs):
    # Linear projections
    Q = self.WQ(inputs)
    K = self.WK(inputs)
    V = self.WV(inputs)

    scores = torch.matmul(Q, K.transpose(-2, -1))
    probs = F.softmax(scores, dim=-1)
    new_embedding = torch.matmul(probs, V)
    # Projection in WO
    new_embedding = self.WO(new_embedding)
    return new_embedding

### Teste das Camadas de Atenção

In [18]:
# Test data
tst_dim = 5
tst_vocab = 10
data = torch.randint(0, tst_vocab, (tst_dim,))
embedding = nn.Embedding(tst_vocab, tst_dim)
embeds = embedding(data)

# Projections (need to be the same for this test)
WQ = nn.Linear(tst_dim, tst_dim)
WK = nn.Linear(tst_dim, tst_dim)
WV = nn.Linear(tst_dim, tst_dim)
WO = nn.Linear(tst_dim, tst_vocab)

# Loop
attn_loop = SelfAttention_Loop(WQ, WK, WV, WO)
embeds_attn_loop = attn_loop(embeds)
# Matrix
attn_matrix = SelfAttention_Matrix_tst(WQ, WK, WV, WO)
embeds_attn_matrix = attn_matrix(embeds)

print("Loop Embeds:")
print(embeds_attn_loop)

print("Matrix Embeds:")
print(embeds_attn_matrix)

# Check results
print()
print(f'Loop and Matrix results are the same: {torch.allclose(embeds_attn_loop, embeds_attn_matrix)}')

Loop Embeds:
tensor([[-0.2173,  0.2699,  0.6123, -0.2549, -0.2727,  0.1416,  0.3101,  0.0980,
          0.1448,  0.2116],
        [-0.1707,  0.2223,  0.5366, -0.1959, -0.1052,  0.0558,  0.2751,  0.0727,
          0.0592,  0.1493],
        [-0.1075,  0.1278,  0.3656, -0.1041,  0.0171, -0.0250,  0.2560,  0.1537,
          0.0122,  0.1725],
        [-0.1750,  0.2162,  0.5186, -0.1977, -0.1627,  0.0788,  0.2909,  0.1181,
          0.0941,  0.1961],
        [-0.1075,  0.1278,  0.3656, -0.1041,  0.0171, -0.0250,  0.2560,  0.1537,
          0.0122,  0.1725]], grad_fn=<StackBackward0>)
Matrix Embeds:
tensor([[-0.2173,  0.2699,  0.6123, -0.2549, -0.2727,  0.1416,  0.3101,  0.0980,
          0.1448,  0.2116],
        [-0.1707,  0.2223,  0.5366, -0.1959, -0.1052,  0.0558,  0.2751,  0.0727,
          0.0592,  0.1493],
        [-0.1075,  0.1278,  0.3656, -0.1041,  0.0171, -0.0250,  0.2560,  0.1537,
          0.0122,  0.1725],
        [-0.1750,  0.2162,  0.5186, -0.1977, -0.1627,  0.0788,  0.2909,  

In [19]:
class BengioModel_SelfAttentionMatrix(torch.nn.Module):
    def __init__(self, vocab_size, embedding_dim, context_size, h):
        super(BengioModel_SelfAttentionMatrix, self).__init__()
        self.embeddings = nn.Embedding(vocab_size+1, embedding_dim)
        self.attention = SelfAttention_Matrix(embedding_dim)        
        self.linear1 = nn.Linear(context_size * embedding_dim, h)
        self.relu = torch.nn.ReLU()
        self.linear2 = nn.Linear(h, vocab_size+1)
        # Softmax to scale outputs
        self.logSoftMax = torch.nn.LogSoftmax(dim=1)

    def forward(self, inputs):
        embeds = self.embeddings(inputs)
        x = torch.stack(torch.unbind(embeds, dim=1), dim=1)
        # Camada de autoatenção
        attention  = self.attention(x)
        # Flatten embeddings
        embeds = embeds.view(attention.size(0), -1)
        # Linear layer
        out = self.linear1(embeds)
        out = self.relu(out)
        # Second layer
        out = self.linear2(out)
        # Softmax output
        out = self.logSoftMax(out)
        return out

In [20]:
class BengioModel(torch.nn.Module):
    def __init__(self, vocab_size, embedding_dim, context_size, h):
        super(BengioModel, self).__init__()
        self.embeddings = nn.Embedding(vocab_size+1, embedding_dim)
        self.linear1 = nn.Linear(context_size * embedding_dim, h)
        self.relu = torch.nn.ReLU()
        self.linear2 = nn.Linear(h, vocab_size+1)
        # Softmax to scale outputs
        self.logSoftMax = torch.nn.LogSoftmax(dim=1)

    def forward(self, inputs):
        embeds = self.embeddings(inputs)
        # Flatten embeddings
        embeds = embeds.view(embeds.size(0), -1)
        # Linear layer
        out = self.linear1(embeds)
        out = self.relu(out)
        # Second layer
        out = self.linear2(out)
        # Softmax output
        out = self.logSoftMax(out)
        return out

## Funções de Treinamento e Avaliação do Modelo

In [21]:
def count_parameters(model):
    total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print(f'The model has a total of {total_params:,} parameters.')

In [22]:
def initial_eval(model):
    # Initial Perplexity and Loss
    # Before training
    model.eval()

    loss = 0
    perp = 0

    with torch.no_grad():
        for inputs, targets in train_loader:
            inputs = inputs.to(device)
            targets = targets.to(device)
            outputs = model(inputs)
            loss += criterion(outputs, targets).item()

    loss /= len(train_loader)
    perp = torch.exp(torch.tensor(loss))

    print(f'Initial Loss: {loss:.4f}')
    print(f'Initial Perplexity: {perp:.4f}')

In [23]:
def train(model, criterion, optimizer):
      # Training Loop
      model.train()
      for epoch in range(epochs):

            epoch_start = time.time()
            # Metrics
            epoch_loss = 0
            epoch_correct = 0
            epoch_samples = 0

            for inputs, targets in train_loader:
                  inputs = inputs.to(device)  # Move input data to the device
                  targets = targets.to(device)

                  # Forward pass
                  outputs = model(inputs)
                  loss = criterion(outputs, targets)

                  # Backward pass and optimization
                  optimizer.zero_grad()
                  loss.backward()
                  optimizer.step()

                  # Loss
                  epoch_loss += loss.item()

                  # Predicted
                  _, predicted = torch.max(outputs, 1)
                  epoch_correct += (predicted == targets).sum().item()
                  epoch_samples += targets.size(0)

            # Calculate average loss and accuracy for epoch
            avg_loss = epoch_loss / len(train_loader)
            acc = epoch_correct / epoch_samples

            # Perplexity
            perp = torch.exp(torch.tensor(avg_loss))

            epoch_end = time.time()
            epoch_time = epoch_end - epoch_start
            # Print epoch statistics
            print(f'Epoch [{epoch+1}/{epochs}], Time:{epoch_time:.2f}, Loss: {avg_loss:.4f}, Accuracy: {acc:.2f}%, Perplexity: {perp:.4f}')


In [24]:
def eval(model, criterion):
    model.eval()

    loss_sum = 0
    total_sum = 0
    correct_sum = 0
    eval_round = 0

    loss = 0
    perp = 0

    with torch.no_grad():
        for inputs, targets in val_loader:
            inputs = inputs.to(device)
            targets = targets.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, targets)      
            loss_sum += loss

            # Get the predicted labels
            _, predicted = torch.max(outputs, 1)

            total_sum += targets.size(0)
            correct_sum += (predicted == targets).sum().item()
            eval_round += 1

    # Calculate accuracy
    acc = 100 * correct_sum / total_sum

    # Calculate average perplexity
    average_loss = loss_sum / len(val_loader)
    average_perplexity = torch.exp(average_loss)

    print(f'Test Accuracy: {acc:.2f}%')
    print(f'Average Loss: {average_loss:.2f}')
    print(f'Average Perplexity: {average_perplexity:.2f}')

In [25]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

## 1. Sem camada de atenção

In [26]:
model = BengioModel(vocab_size, embedding_dim, context_size, hidden_dim)
print("Model without Self Attention:")
print()
count_parameters(model)

# Cross Entropy
criterion = nn.CrossEntropyLoss()

# Optimizer
optimizer = torch.optim.SGD(model.parameters(), lr)

model.to(device)

print()
print("Training Start")
print()
train(model, criterion, optimizer)

print()
print("Evaluation Start")
print()
eval(model, criterion)

Model without Self Attention:

The model has a total of 2,089,353 parameters.

Training Start

Epoch [1/10], Time:5.64, Loss: 6.4502, Accuracy: 0.11%, Perplexity: 632.8126
Epoch [2/10], Time:2.82, Loss: 5.5790, Accuracy: 0.15%, Perplexity: 264.8025
Epoch [3/10], Time:2.55, Loss: 5.1947, Accuracy: 0.18%, Perplexity: 180.3141
Epoch [4/10], Time:2.73, Loss: 4.9040, Accuracy: 0.20%, Perplexity: 134.8277
Epoch [5/10], Time:2.87, Loss: 4.6603, Accuracy: 0.22%, Perplexity: 105.6705
Epoch [6/10], Time:3.10, Loss: 4.4451, Accuracy: 0.24%, Perplexity: 85.2074
Epoch [7/10], Time:2.96, Loss: 4.2440, Accuracy: 0.26%, Perplexity: 69.6866
Epoch [8/10], Time:2.97, Loss: 4.0530, Accuracy: 0.28%, Perplexity: 57.5683
Epoch [9/10], Time:3.03, Loss: 3.8716, Accuracy: 0.30%, Perplexity: 48.0209
Epoch [10/10], Time:3.02, Loss: 3.6953, Accuracy: 0.32%, Perplexity: 40.2578

Evaluation Start

Test Accuracy: 19.11%
Average Loss: 5.32
Average Perplexity: 204.95


## 2. Com camada de atenção

In [29]:
model_attn = BengioModel_SelfAttentionMatrix(vocab_size, embedding_dim, context_size, hidden_dim)
print("Model with Self Attention:")
print()
count_parameters(model_attn)

# Cross Entropy
criterion = nn.CrossEntropyLoss()

# Optimizer
optimizer = torch.optim.SGD(model_attn.parameters(), lr)

model_attn.to(device)

print()
print("Training Start")
print()
train(model_attn, criterion, optimizer)

print()
print("Evaluation Start")
print()
eval(model_attn, criterion)

Model with Self Attention:

The model has a total of 3,571,729 parameters.

Training Start

Epoch [1/10], Time:3.50, Loss: 6.4457, Accuracy: 0.11%, Perplexity: 630.0162
Epoch [2/10], Time:3.28, Loss: 5.5807, Accuracy: 0.15%, Perplexity: 265.2518
Epoch [3/10], Time:3.29, Loss: 5.1977, Accuracy: 0.18%, Perplexity: 180.8607
Epoch [4/10], Time:3.14, Loss: 4.9091, Accuracy: 0.20%, Perplexity: 135.5202
Epoch [5/10], Time:3.25, Loss: 4.6662, Accuracy: 0.22%, Perplexity: 106.2960
Epoch [6/10], Time:3.30, Loss: 4.4488, Accuracy: 0.24%, Perplexity: 85.5281
Epoch [7/10], Time:3.22, Loss: 4.2458, Accuracy: 0.26%, Perplexity: 69.8131
Epoch [8/10], Time:3.24, Loss: 4.0557, Accuracy: 0.28%, Perplexity: 57.7236
Epoch [9/10], Time:3.28, Loss: 3.8739, Accuracy: 0.30%, Perplexity: 48.1275
Epoch [10/10], Time:3.14, Loss: 3.6979, Accuracy: 0.32%, Perplexity: 40.3609

Evaluation Start

Test Accuracy: 19.32%
Average Loss: 5.32
Average Perplexity: 203.83
